# Model: Desperado v0

---
# Load data & libraries

In [1]:
# TOP VERSION
MODEL_NAME = "desperado"
VERSION = 0

In [2]:
# import dependencies
import pandas as pd
from numerapi import NumerAPI
from utils.api_keys import PUBLIC_ID, SECRET_KEY

In [3]:
# instantiate numerai api
napi = NumerAPI(public_id=PUBLIC_ID, secret_key=SECRET_KEY)
current_round = napi.get_current_round()
print(f"Current round #: {current_round}")

Current round #: 315


In [6]:
# read val predictions from models
foxhound_val = pd.read_csv(f"predictions/dh_foxhound_v0_val_preds.csv")
deadcell_val = pd.read_csv(f"predictions/dh_deadcell_v0_val_preds.csv")
cobra_val = pd.read_csv(f"predictions/dh_cobra_v0_val_preds.csv")
beautybeast_val = pd.read_csv(f"predictions/dh_beautybeast_v0_val_preds.csv")

print(f"""
    Foxhound predictions   : {len(foxhound_val)}
    Deadcell pedictions    : {len(deadcell_val)}
    Cobra pedictions       : {len(cobra_val)}
    BeautyBeast pedictions : {len(beautybeast_val)}
    """)


    Foxhound predictions   : 2203644
    Deadcell pedictions    : 2203644
    Cobra pedictions       : 2203644
    BeautyBeast pedictions : 2203644
    


In [7]:
# read live predictions from models
foxhound_live = pd.read_csv(f"predictions/dh_foxhound_v0_live_preds_{current_round}.csv")
deadcell_live = pd.read_csv(f"predictions/dh_deadcell_v0_live_preds_{current_round}.csv")
cobra_live = pd.read_csv(f"predictions/dh_cobra_v0_live_preds_{current_round}.csv")
beautybeast_live = pd.read_csv(f"predictions/dh_beautybeast_v0_live_preds_{current_round}.csv")

print(f"""
    Foxhound predictions   : {len(foxhound_live)}
    Deadcell pedictions    : {len(deadcell_live)}
    Cobra pedictions       : {len(cobra_live)}
    BeautyBeast pedictions : {len(beautybeast_live)}
    """)


    Foxhound predictions   : 5296
    Deadcell pedictions    : 5296
    Cobra pedictions       : 5296
    BeautyBeast pedictions : 5296
    


---
# Create ensemble prediction

In [8]:
# model name
model_name = f"dh_{MODEL_NAME}_v{VERSION}"
model_name

'dh_desperado_v0'

In [11]:
# average val preds
desperado_val = foxhound_val.merge(
    right=deadcell_val, how='inner', on="id", suffixes=('', '2')).merge(
    right=cobra_val, how='inner', on="id", suffixes=('', '3')).merge(
    right=beautybeast_val, how='inner', on="id", suffixes=('', '4'))

desperado_val.columns = ["id", "foxhound", "deadcell", "cobra", "beautybeast"]
desperado_val["prediction"] = (
    desperado_val["foxhound"] + 
    desperado_val["deadcell"] + 
    desperado_val["cobra"] +
    desperado_val["beautybeast"]
    ) / 4
desperado_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2203644 entries, 0 to 2203643
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           object 
 1   foxhound     float64
 2   deadcell     float64
 3   cobra        float64
 4   beautybeast  float64
 5   prediction   float64
dtypes: float64(5), object(1)
memory usage: 117.7+ MB


In [12]:
# average live preds
desperado_live = foxhound_live.merge(
    right=deadcell_live, how='inner', on="id", suffixes=('', '2')).merge(
    right=cobra_live, how='inner', on="id", suffixes=('', '3')).merge(
    right=beautybeast_live, how='inner', on="id", suffixes=('', '4'))

desperado_live.columns = ["id", "foxhound", "deadcell", "cobra", "beautybeast"]
desperado_live["prediction"] = (
    desperado_live["foxhound"] + 
    desperado_live["deadcell"] + 
    desperado_live["cobra"] +
    desperado_live["beautybeast"]
    ) / 4
desperado_live.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5296 entries, 0 to 5295
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           5296 non-null   object 
 1   foxhound     5296 non-null   float64
 2   deadcell     5296 non-null   float64
 3   cobra        5296 non-null   float64
 4   beautybeast  5296 non-null   float64
 5   prediction   5296 non-null   float64
dtypes: float64(5), object(1)
memory usage: 289.6+ KB


In [13]:
# check val file
desperado_val.head()

,id,foxhound,deadcell,cobra,beautybeast,prediction
0,n000101811a8a843,0.129369,0.083149,0.009770,0.113878,0.084041
1,n001e1318d5072ac,0.439908,0.511116,0.763217,0.829601,0.635960
2,n002a9c5ab785cbb,0.834553,0.442237,0.684775,0.823740,0.696326
3,n002ccf6d0e8c5ad,0.929264,0.899298,0.881232,0.997238,0.926758
4,n0051ab821295c29,0.169530,0.265021,0.035853,0.142973,0.153345


In [14]:
# check live file
desperado_live.head()

,id,foxhound,deadcell,cobra,beautybeast,prediction
0,n00145e088eeee46,0.777379,0.599887,0.797961,0.508308,0.670884
1,n002049bbe13ce5b,0.032477,0.058346,0.063255,0.026246,0.045081
2,n00260dbcadb07ea,0.081760,0.047961,0.087991,0.325340,0.135763
3,n0029a05645f09d4,0.902568,0.712613,0.756420,0.723376,0.773744
4,n002fbd4f847eb9a,0.511896,0.477530,0.234705,0.955627,0.544940


---
# Save predictions

In [15]:
# save ensemble preds
desperado_val = desperado_val[["id", "prediction"]].set_index("id")
desperado_live = desperado_live[["id", "prediction"]].set_index("id")

desperado_val.to_csv(f"predictions/{model_name}_val_preds.csv")
desperado_live.to_csv(f"predictions/{model_name}_live_preds_{current_round}.csv")